**Домашнее задание 2. Извлечение коллокаций.**

Описание задания:
Скачайте корпус текстов, обработайте его с помощью UDPipe, извлеките все группы "глагол + прямое дополнение, выраженное существительным" (не учитывайте глаголы, которые встречаются в корпусе менее 50 раз).

Оцените полученные словосочетания следующими метриками: log-likelihood, dice, PMI (можно использовать nltk.collocations).

Подготовьте "золотой стандарт" коллокаций (далее ЗС) для этого корпуса: возьмите словосочетания, которые попадают в топ-100 по всем метрикам, пересеките со словарем глагольной сочетаемости.

Добавьте в ЗС словосочетания из топ-100 , которые не вошли в словарь, но являются коллокациями (если такие есть), объясните свой выбор.

Оцените ранговую корреляцию (коэффициент Спирмена) результатов по каждой метрике с ЗС. Как это работает, читайте, например, тут. Можно использовать scipy.stats.spearmanr. Опишите ошибки каждой метрики.

1.Обработка корпуса

In [1]:
!C:/Users/Дарья/nlp/udpipe-1.2.0-bin/bin-win64/udpipe --input horizontal --output conllu \
--tokenize --tag --parse \
C:/Users/Дарья/nlp/russian-syntagrus-ud-2.4-190531.udpipe \
< testset2.txt > testset2.conllu

Loading UDPipe model: done.


In [2]:
from nltk.parse import DependencyGraph
trees = []

with open('testset2.conllu', encoding = 'utf-8') as f:
    parsed_sents = f.read().split('\n\n')

    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))

2.Создадим список частотных глаголов

In [31]:
verbs = []
lemmas = []
for t in trees:
    try:
        g = DependencyGraph(t, top_relation_label='root')
    except:
        continue
    for n in g.nodes:
        if g.nodes[n]['ctag'] == 'VERB':
            verbs.append(g.nodes[n]['lemma'])
        if g.nodes[n]['ctag'] != 'PUNCT':
            if g.nodes[n]['lemma'] != None:
                lemmas.append(g.nodes[n]['lemma'])

In [35]:
verbs[:5]

['предложить', 'объединить', 'заявить', 'прийтись', 'забыть']

In [36]:
lemmas[:5]

['предложить', 'я', 'бы', 'единый', 'Россия']

In [6]:
from collections import Counter

freq_verbs = {}

v_count = Counter(verbs)

for i in v_count.elements():
    if v_count[i] >= 50:
        if i in freq_verbs.keys():
            freq_verbs[i] += 1
        else:
            freq_verbs[i] = 1

In [10]:
freq_verbs

{'заявить': 258,
 'подать': 274,
 'просить': 143,
 'признать': 343,
 'стать': 213,
 'сообщить': 216,
 'обратиться': 139,
 'отказаться': 117,
 'рассматривать': 99,
 'рассказать': 54,
 'рассмотреть': 90,
 'мочь': 349,
 'начаться': 74,
 'обвинить': 461,
 'решить': 86,
 'пытаться': 89,
 'использовать': 56,
 'напомнить': 97,
 'получить': 125,
 'передать': 68,
 'направить': 105,
 'удаться': 61,
 'сказать': 98,
 'иметь': 94,
 'делать': 77,
 'обжаловать': 102,
 'отказать': 59,
 'говорить': 139,
 'счесть': 55,
 'быть': 136,
 'оставить': 61,
 'дать': 70,
 'удовлетворить': 173,
 'вынести': 152,
 'требовать': 172,
 'являться': 113,
 'обвинять': 119,
 'принять': 151,
 'выплатить': 59,
 'назначить': 63,
 'постановить': 54,
 'обязать': 58,
 'принадлежать': 54,
 'подтвердить': 91,
 'находиться': 117,
 'арестовывать': 82,
 'доказать': 56,
 'пояснить': 58,
 'предъявить': 97,
 'согласиться': 62,
 'взыскать': 62,
 'отклонить': 69,
 'провести': 63,
 'приговорить': 122,
 'запретить': 55,
 'считать': 152,
 '

3.Вытащим пары verb + noun.obj

In [19]:
vn_phrases = []
for t in trees:
    try:
        g = DependencyGraph(t, top_relation_label='root')
        for t in g.triples():
            if t[0][1] == 'VERB' and t[1] == 'obj' and t[2][1] == 'NOUN':
                if t[0][0] in freq_verbs:
                    phrase = [t[0][0], t[2][0]]
                    vn_phrases.append(phrase)
    except:
        continue

In [20]:
vn_phrases[:10]

[['признать', 'договор'],
 ['вынести', 'приговор'],
 ['признать', 'ответственность'],
 ['выплатить', 'компенсации'],
 ['обжаловать', 'санкцию'],
 ['удовлетворить', 'жалобу'],
 ['взыскать', 'деньги'],
 ['обжаловать', 'санкцию'],
 ['оспаривать', 'решение'],
 ['оспаривать', 'решение']]

4.Оценим полученные словосочетания метриками

a) likelihood

In [42]:
import nltk
from nltk import collocations
from nltk.collocations import *

In [97]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(vn_phrases)

In [98]:
likeli_scores_best = finder.nbest(collocations.BigramAssocMeasures().likelihood_ratio, 100)

In [99]:
likeli_scores_best[:5]

[('обжаловать', 'решение'),
 ('передать', 'дело'),
 ('запретить', 'деятельность'),
 ('назначить', 'наказание'),
 ('рассмотреть', 'вопрос')]

b) Dice

In [100]:
dice_scores_best = finder.nbest(collocations.BigramAssocMeasures().dice, 100)

In [101]:
dice_scores_best[:5]

[('делать', 'вид'),
 ('объявить', 'перерыв'),
 ('отказать', 'жалобы'),
 ('сообщить', 'сведения'),
 ('стать', 'обыски')]

c) PMI

In [102]:
pmi_scores_best = finder2.nbest(collocations.BigramAssocMeasures().pmi, 100)
pmi_scores_best[:5]

[('делать', 'вид'),
 ('отказать', 'жалобы'),
 ('сообщить', 'сведения'),
 ('стать', 'обыски'),
 ('арестовывать', 'счета')]

5.Подготовим золотой стандарт

In [110]:
intersection = list(set(likeli_scores_best) & set(dice_scores_best) & set(pmi_scores_best))
intersection[:5]

[('назначить', 'наказание'),
 ('пройти', 'прения'),
 ('направить', 'документы'),
 ('доказать', 'незаконность'),
 ('объявить', 'перерыв')]

In [256]:
len(intersection)

52

In [124]:
with open("verb_coll.txt", "r", encoding='utf-8') as d:
    text = d.read()

In [200]:
colls = []
lines = text.split('\n')
for i in lines:
    tri = i.split('\t')
    if tri != ['']:
        colls.append(tri[2])

In [203]:
colls[:5]

['аборт делать',
 'делать аборт',
 'сделать аборт',
 'авария произошла',
 'авария случилась']

In [202]:
len(colls)

9729

In [245]:
new_colls = []
for i in colls:
    words = i.split()
    pos = []
    for w in words:
        p = morph.parse(w)[0]
        tags = str(p.tag).split(',')
        pos.append(tags[0])
    if pos[-1] == 'NOUN':
        pair = []
        for j in i.split():
            pp = morph.parse(j)[0]
            token = pp.normal_form
            pair.append(token)
        s = tuple(pair)
        new_colls.append(s)

In [246]:
new_colls[:5]

[('делать', 'аборт'),
 ('сделать', 'аборт'),
 ('потерпеть', 'авария'),
 ('произойти', 'авария'),
 ('случиться', 'авария')]

In [247]:
len(new_colls)

7688

In [254]:
gold_stand = set(new_colls) & set(intersection)

In [255]:
gold_stand

{('делать', 'вид'),
 ('объявить', 'перерыв'),
 ('передать', 'дело'),
 ('предъявить', 'обвинение'),
 ('пройти', 'курс')}

6.Добавим коллокации

In [262]:
import random

In [263]:
dop_set = random.sample(set(intersection).difference(gold_stand), 10)

In [264]:
dop_set

[('пройти', 'этап'),
 ('решить', 'директора'),
 ('иметь', 'резонанс'),
 ('считать', 'репутацию'),
 ('оспаривать', 'предписания'),
 ('взыскать', 'задолженность'),
 ('запретить', 'ЗАО'),
 ('иметь', 'повод'),
 ('оставить', 'заявление'),
 ('доказать', 'незаконность')]

In [266]:
new_gs = gold_stand | set(dop_set)

In [267]:
new_gs

{('взыскать', 'задолженность'),
 ('делать', 'вид'),
 ('доказать', 'незаконность'),
 ('запретить', 'ЗАО'),
 ('иметь', 'повод'),
 ('иметь', 'резонанс'),
 ('объявить', 'перерыв'),
 ('оспаривать', 'предписания'),
 ('оставить', 'заявление'),
 ('передать', 'дело'),
 ('предъявить', 'обвинение'),
 ('пройти', 'курс'),
 ('пройти', 'этап'),
 ('решить', 'директора'),
 ('считать', 'репутацию')}